In [1]:
!pip install -U langgraph langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 9.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.27
    Uninstalling langchain-0.3.27:
      Successfully uninstalled langchain-0.3.27


The API Key should be named "OPENAI_API_KEY", while creating the key (https://openrouter.ai/settings/keys)

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

False

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

In [5]:
from typing_extensions import TypedDict

class CounterState(TypedDict):
    count: int

In [6]:
def increment(state: CounterState) -> dict:
    state["count"] += 1
    return state

In [7]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(CounterState)

builder.add_node("increment", increment)

# Define the execution order: START -> increment -> END
builder.add_edge(START, "increment")
builder.add_edge("increment", END)

graph = builder.compile()

In [8]:
initial_state: CounterState = {"count": 0}

result = graph.invoke(initial_state)

print(result)

# Output
# {'count': 1}

{'count': 1}


In [31]:
from typing import Literal

def should_continue(state: CounterState) -> Literal["increment", END]:

    if state["count"] < 10: # keep looping
        print("count: ", state["count"])
        return "increment"


    return END # stop the graph

In [32]:
builder = StateGraph(CounterState)

builder.add_node("increment", increment)

builder.add_edge(START, "increment")

builder.add_conditional_edges(
    "increment",
    should_continue,
    ["increment", END],
)

graph = builder.compile()

result = graph.invoke({"count": 0})

print(result)

# Output
# {'count': 3}

count:  1
count:  2
count:  3
count:  4
count:  5
count:  6
count:  7
count:  8
count:  9
{'count': 10}


In [30]:
from typing_extensions import TypedDict

class AgentState(TypedDict):
    user_input: str
    response: str

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="openai/gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
)

In [15]:
from langchain_core.messages import SystemMessage, HumanMessage

def llm_node(state: AgentState) -> dict:
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=state["user_input"]),
    ]

    reply = llm.invoke(messages)

    return {"response": reply.content}

In [16]:
from langgraph.graph import StateGraph, START, END

builder = StateGraph(AgentState)

builder.add_node("llm", llm_node)

# define the flow: START -> llm -> END
builder.add_edge(START, "llm")
builder.add_edge("llm", END)

graph = builder.compile()

In [17]:
initial_state: AgentState = {
    "user_input": "What is GIL in python?",
    "response": "",
}

result = graph.invoke(initial_state)

print("User:", initial_state["user_input"])
print("Assistant:", result["response"])

User: What is GIL in python?
Assistant: GIL, or Global Interpreter Lock, is a mechanism used in CPython (the standard implementation of Python) to ensure that only one thread executes Python bytecode at a time. This lock is necessary because Python's memory management is not thread-safe; access to Python objects and internal data structures must be synchronized to prevent corruption.

### Key Points about GIL:

1. **Thread Safety**: The GIL simplifies memory management and ensures thread safety for Python objects, avoiding the complexities of multi-threading issues like race conditions, which can arise without a locking mechanism.

2. **Performance Implications**: Due to the GIL, even in multi-threaded Python programs, only one thread can execute Python code at a time. This means that while I/O-bound tasks can benefit from threading (where the GIL can be released during I/O operations), CPU-bound tasks may not see performance improvements with threads, as they cannot execute in paralle

In [18]:
state2: AgentState = {
    "user_input": "Can you summarise that in two lines?",
    "response": "",
}
result2 = graph.invoke(state2)

print("\nTurn 2 - User:", state2["user_input"])
print("Turn 2 - Assistant:", result2["response"])


Turn 2 - User: Can you summarise that in two lines?
Turn 2 - Assistant: Sure! Please provide the text you would like summarized.


In [19]:
from typing_extensions import TypedDict, Annotated
from operator import add
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver

class State(TypedDict):
    foo: str
    bar: Annotated[list[str], add]

def node_a(state: State):
    # overwrite foo, append "a" to bar
    return {"foo": "a", "bar": ["a"]}

def node_b(state: State):
    # overwrite foo, append "b" to bar
    return {"foo": "b", "bar": ["b"]}

In [20]:
builder = StateGraph(State)
builder.add_node("node_a", node_a)
builder.add_node("node_b", node_b)

builder.add_edge(START, "node_a")
builder.add_edge("node_a", "node_b")
builder.add_edge("node_b", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [21]:
config = {"configurable": {"thread_id": "1"}}

final_state = graph.invoke({"foo": "", "bar": []}, config=config)
print("Final state:", final_state)

# Output
# Final state: {'foo': 'b', 'bar': ['a', 'b']}

Final state: {'foo': 'b', 'bar': ['a', 'b']}


In [22]:
history = list(graph.get_state_history(config))

for i, snap in enumerate(history[::-1]):
    print(f"\nCheckpoint {i}:")
    print("  created_at:", snap.created_at)
    print("  node:", snap.metadata)
    print("  values:", snap.values)


Checkpoint 0:
  created_at: 2025-11-20T21:55:24.450980+00:00
  node: {'source': 'input', 'step': -1, 'parents': {}}
  values: {'bar': []}

Checkpoint 1:
  created_at: 2025-11-20T21:55:24.454246+00:00
  node: {'source': 'loop', 'step': 0, 'parents': {}}
  values: {'foo': '', 'bar': []}

Checkpoint 2:
  created_at: 2025-11-20T21:55:24.456344+00:00
  node: {'source': 'loop', 'step': 1, 'parents': {}}
  values: {'foo': 'a', 'bar': ['a']}

Checkpoint 3:
  created_at: 2025-11-20T21:55:24.459949+00:00
  node: {'source': 'loop', 'step': 2, 'parents': {}}
  values: {'foo': 'b', 'bar': ['a', 'b']}


In [23]:
from typing_extensions import TypedDict, Annotated
from langchain_core.messages import AnyMessage
import operator

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [24]:
from langchain_core.messages import SystemMessage

def chat_llm_node(state: MessagesState) -> dict:

    # build prompt from system + existing conversation
    history = [SystemMessage(content="You are a helpful assistant.")]
    history.extend(state["messages"])

    reply = llm.invoke(history)

    return {"messages": [reply]}

In [25]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage

checkpointer = InMemorySaver()

builder = StateGraph(MessagesState)
builder.add_node("chat_llm", chat_llm_node)
builder.add_edge(START, "chat_llm")
builder.add_edge("chat_llm", END)

graph = builder.compile(checkpointer=checkpointer)

In [26]:
config = {"configurable": {"thread_id": "user_123"}}


In [27]:
config = {"configurable": {"thread_id": "user_123"}}

# Turn 1
state1 = {"messages": [HumanMessage(content="What is GIL in python?")]}
result1 = graph.invoke(state1, config=config)

for m in result1["messages"]:
    print(type(m).__name__, ":", m.content)

# Turn 2
state2 = {"messages": [HumanMessage(content="Summarise it in two lines")],}
result2 = graph.invoke(state2, config=config)

for m in result2["messages"][-2:]:
    print(type(m).__name__, ":", m.content)

HumanMessage : What is GIL in python?
AIMessage : GIL stands for Global Interpreter Lock. It is a mechanism used in Python (specifically CPython, the most common implementation of Python) to ensure that only one thread executes Python bytecode at a time. Here are some key points about GIL:

1. **Thread Safety**: The GIL makes it easier to manage memory and ensures thread safety with respect to Python's internal data structures. It prevents race conditions when multiple threads interact with Python objects.

2. **Concurrency Limitation**: While the GIL simplifies development in multithreaded programs, it can also be a limitation for CPU-bound programs. When CPU-bound tasks are executed in multiple threads, the GIL forces one thread to run at a time, which can lead to suboptimal performance compared to processes that can run truly concurrently.

3. **I/O-Bound vs CPU-Bound**: The GIL is less of an issue for I/O-bound programs (those that spend a lot of time waiting for external events, l